# **Code for Training**

In [24]:
import numpy as np  # to load dataset
import math
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.optim
import os

#%matplotlib inline

In [25]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device in use: {device}")

Device in use: cpu


## For Google Colab

In [26]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)

# # linux command
# %cd /content/drive/MyDrive/HapticData/

## Load the dataset

In [27]:
# define the dataset
# 0 corresponds to rotating
# 1 corresponds to grasping

# num rows for one grasp (24 rows = 1 second)
block_size = 24

def normalize_vector(nd_arr: np.ndarray) -> np.ndarray:
    vector_length = sum([i ** 2 for i in nd_arr]) ** 0.5
    normalized_arr = [i / vector_length for i in nd_arr]
    return normalized_arr

# Rotation
unedited_rotating_dataset = np.loadtxt('rotating.csv', delimiter=",")
# Split into multiple movements over time ("block_size" seconds)
edited_rotating_dataset = unedited_rotating_dataset[:,1:21] #corresponding to cols 2 - 21 inclusive, excluding the last row of zeros
# edited_rotating_dataset = np.delete(edited_rotating_dataset, 5, 1)
# Convert positions into velocities
edited_rotating_dataset = np.diff(edited_rotating_dataset, axis=0)
# Normalize each row
edited_rotating_dataset = np.array([normalize_vector(row) for row in edited_rotating_dataset])

# Grasping
unedited_grasping_dataset = np.loadtxt('grasping.csv', delimiter=",")
edited_grasping_dataset = unedited_grasping_dataset[:,1:21]
# edited_grasping_dataset = np.delete(edited_grasping_dataset, 5, 1)
edited_grasping_dataset = np.diff(edited_grasping_dataset, axis=0)
# Normalize
edited_grasping_dataset = np.array([normalize_vector(row) for row in edited_grasping_dataset])

# Process data

In [28]:
columns = edited_rotating_dataset.shape[1]

# Rotating
rows = edited_rotating_dataset.shape[0]
X = np.zeros((rows - block_size + 1, block_size * columns))  # number of rows times columns rows (from flattening)

# For each block
for i in range(0, rows - block_size + 1):
  # Get entire row, 1st block_size * 24 rows, 2nd of them and so on..
  X[i] = np.ndarray.flatten(edited_rotating_dataset[i : i + block_size, :])

# X now contains mutliple blocks
y = np.zeros((rows - block_size + 1))


# grasping
rows = edited_grasping_dataset.shape[0]
X2 = np.zeros((rows - block_size + 1, block_size * columns))
# For each block
for i in range(0, rows - block_size + 1):
  # Get entire row, 1st, block_size * 24 rows, second of them and so on..
  X2[i] = np.ndarray.flatten(edited_grasping_dataset[i : i + block_size, :])
# X now contains mutliple blocks
y2 = np.ones((rows - block_size + 1))
combined_X = np.concatenate((X, X2), axis=0)
combined_y = np.concatenate((y, y2), axis=0)

In [29]:
columns

20

## Plotting

In [30]:
# column_names = [i + 1 for i in range(20)]
# graph_combined_X = pd.DataFrame(combined_X)
# graph_combined_X['labels'] = combined_y
# sns.pairplot(graph_combined_X, vars=column_names, hue="labels")

# # remove these columns
# highly_correlated_cols = [] # put column numbers here
# combined_X = np.delete(combined_X, highly_correlated_cols, axis=1)

## Prepare for Processing

In [31]:
# Split training and testing data
X_train, X_test, y_train, y_test = train_test_split(combined_X, combined_y, test_size=0.3, random_state=42)

# covnert into tensors since numpy uses 64 bit floating point
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
y_train = torch.reshape(y_train,(-1,1)) # a single dimension uses -1 to infer the length. new shape
y_test = torch.reshape(y_test,(-1,1)) # a single dimension uses -1 to infer the length. new shape

print(f"Total data size: {combined_X.shape[0]}")
print(f"Train data size: {X_train.shape[0]}")
print(f"Test data size:  {X_test.shape[0]}")

Total data size: 3430
Train data size: 2401
Test data size:  1029


In [32]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(block_size * columns, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

print(f"input size {block_size * columns}")

input size 480


In [33]:
model = NeuralNetwork().to(device)
X_train.to(device)
y_train.to(device)
X_test.to(device)
y_test.to(device)

tensor([[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [1.],
        [1.]])

## Preparation for training

In [34]:
loss_fn = nn.BCELoss() # used for binary classification problems
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Training

In [35]:
n_epochs = 15
batch_size = 4

# CPU
if device == "cpu":
  for epoch in range(n_epochs):
    for i in range(0, len(X) - batch_size, batch_size):
      Xbatch = X_train[i:i+batch_size] # specifying the rows
      y_pred = model(Xbatch) # scalar
      ybatch = y_train[i:i+batch_size]
      loss = loss_fn(y_pred, ybatch)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print(f"epoch: {epoch + 1}/{n_epochs}", end="\r")
  print(f'CPU: Finished epoch {epoch}, latest loss {loss}')

# GPU
elif device == "cuda":
  for epoch in range(n_epochs):
    for i in range(0, len(X) - batch_size, batch_size):
      Xbatch = X_train[i:i+batch_size] # specifying the rows
      y_pred = model(Xbatch.cuda()) # scalar
      ybatch = y_train[i:i+batch_size].cuda()
      loss = loss_fn(y_pred, ybatch)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print(f"epoch: {epoch + 1}/{n_epochs}", end="\r")
    print(f'CUDA: Finished epoch {epoch}, latest loss {loss}')
else:
  print("Not using CPU or CUDA")

CPU: Finished epoch 14, latest loss 2.3154441322171485e-21


In [36]:
# evaluate the accuracy of the model

with torch.no_grad():  # no_grad to avoid differentiating
    y_pred1 = model(X_train.to(device))
    y_pred2 = model(X_test.to(device))


# rounds prediction to nearest integer, check if equal to y
accuracy1 = (y_pred1.round() == y_train.to(device)).float().mean()
accuracy2 = (y_pred2.round() == y_test.to(device)).float().mean()
print(f"Training Data Accuracy: {accuracy1 * 100}%")
print(f"Testing Data Accuracy: {accuracy2 * 100}%")

Training Data Accuracy: 98.70887756347656%
Testing Data Accuracy: 94.94654846191406%


###Save model


In [37]:
# print state dict, a dictionary containing label for each weight/bias and their value
for param_tensor in model.state_dict():
  print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# print state dict for the optimizer, containing hyperparameters
for var_name in optimizer.state_dict():
  print(var_name, "\t", optimizer.state_dict()[var_name])

linear_relu_stack.0.weight 	 torch.Size([256, 480])
linear_relu_stack.0.bias 	 torch.Size([256])
linear_relu_stack.2.weight 	 torch.Size([128, 256])
linear_relu_stack.2.bias 	 torch.Size([128])
linear_relu_stack.4.weight 	 torch.Size([64, 128])
linear_relu_stack.4.bias 	 torch.Size([64])
linear_relu_stack.6.weight 	 torch.Size([1, 64])
linear_relu_stack.6.bias 	 torch.Size([1])
state 	 {0: {'step': tensor(6915.), 'exp_avg': tensor([[ 8.9980e-11, -1.4563e-08, -3.7514e-09,  ...,  4.3938e-09,
          1.2765e-08,  2.0681e-08],
        [-3.9767e-08, -1.9852e-08, -6.9690e-08,  ...,  6.2616e-09,
          5.9518e-09,  7.5887e-08],
        [ 1.7139e-07,  5.8344e-08,  2.1700e-07,  ..., -2.0211e-07,
          1.7526e-08, -1.7581e-07],
        ...,
        [ 7.5802e-09, -5.0852e-08,  6.3128e-07,  ...,  1.0602e-07,
          5.0073e-08, -7.9404e-08],
        [ 1.6447e-10, -3.4353e-09,  2.5596e-07,  ..., -1.6516e-07,
         -2.1539e-08, -7.0915e-08],
        [-2.4766e-11, -2.3616e-09,  9.3062e-

In [38]:
# in google drive, this saves inside the folder that the notebook was initially mounted
PATH = "classifier.pt"

In [39]:
torch.save(model.state_dict(), PATH)

# **Code for testing**

In [40]:
# replace variable with local path in linux
PATH = "classifier.pt"

### Load Existing Model

In [41]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(PATH))
model.eval() # must set dropout and batch normalization layers to evaluation mode

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=480, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [42]:
# import rospy
# from std_msgs.msg import String

# def callback(data):
#     rospy.loginfo(rospy.get_caller_id() + 'I heard %s', data.data)

# def listener():

#     # In ROS, nodes are uniquely named. If two nodes with the same
#     # name are launched, the previous one is kicked off. The
#     # anonymous=True flag means that rospy will choose a unique
#     # name for our 'listener' node so that multiple listeners can
#     # run simultaneously.
#     rospy.init_node('listener', anonymous=True)

#     rospy.Subscriber('/senseglove/0/rh/joint_states', String, callback) # subscribes to this senseglove joints topic

#     # spin() simply keeps python from exiting until this node is stopped
#     rospy.spin()

# if __name__ == '__main__':
#     listener()